# DeepLabV3+ Transfer Learning Training Demo

DeepLabV3+ w/ResNet101 backbone transfer learning training demo
Net pretrained with COCO val2017, then trained with Yamaha-CMU Off-Road Dataset

---

Authors:
- **Nate Haddad** haddad[dot]na[at]northeastern[dot]edu
- **Amit Mulay** mulay[dot]am[at]northeastern[dot]edu

In [ ]:
import os

import torch
from torchvision import transforms, models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import yaml

from utils import get_dataloader, Trainer, YamahaCMUDataset, display_example_pair, freeze_layers

In [ ]:
torch.cuda.is_available()

In [ ]:
np.random.seed(42)

Set up all of our path variables and hyperparamters needed to train the model. Adjust accordingly to your needs

In [ ]:
with open('config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

Set up the `data` and `model` directories

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')
    
if not os.path.exists('models'):
    os.makedirs('models')

---

**IMPORTANT:** we require that you now download the Yamaha CMU Off-Road Dataset from the following url before proceding. The directory and files should be unzipped and placed in the `data` directory of this repository, created above.

**Link to download dataset:** https://theairlab.org/yamaha-offroad-dataset/

---

Load an example image from the training set

In [ ]:
example_image = Image.open('data/yamaha_v0/train/iid000008/rgb.jpg')
example_mask = Image.open('data/yamaha_v0/train/iid000008/labels.png')
image_display = np.array(example_image)
mask_display = np.array(example_mask.convert('RGB'))
display_example_pair(image_display, mask_display)

Load data into dataloaders

In [ ]:
dataloaders = get_dataloader(config['DATA_PATH'],
                             batch_size=config['BATCH_SIZE'],
                             resize_shape=(config['IMG_HEIGHT'],
                             config['IMG_WIDTH']))

### Model Selection

Create the model. Note that the model is pretrained on COCO val2017. Replace the output layer with 8 mask channels and the correct number of nodes for the last layer

ResNet50

In [ ]:
if config['BACKBONE'] is 'RESNET101':
    model = models.segmentation.deeplabv3_resnet101(pretrained=True,
                                                    progress=True,
                                                    aux_loss=True)
    model.classifier = DeepLabHead(2048, config['NUM_MASK_CHANNELS'])
elif config['BACKBONE'] is 'RESNET50':
    model = models.segmentation.deeplabv3_resnet50(pretrained=True,
                                                    progress=True,
                                                    aux_loss=True)
    model.classifier = DeepLabHead(2048, config['NUM_MASK_CHANNELS'])
elif config['BACKBONE'] is 'MOBILENETV3LARGE'
    model = models.segmentation.deeplabv3_mobilenet_v3_large(pretrained=True,
                                                            progress=True,
                                                            aux_loss=True)
    model.classifier = DeepLabHead(960, config['NUM_MASK_CHANNELS'])
else:
    print('Unknown backbone selected in configuration. Please select from RESNET50, RESNET101, or MOBILENETV3LARGE')
    raise ValueError

Freeze earlier layers

In [ ]:
# freeze_layers(model, 0, 14)

In [ ]:
model.train();

Select the optimizer and loss function

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

Train the model

In [ ]:
trainer = Trainer(model, dataloaders, criterion, optimizer, num_epochs=config['NUM_EPOCHS'], is_inception=True);

---
Save the entire model

In [ ]:
torch.save(model, config['SAVE_MODEL_PATH'])